<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/extract_imagery/01_Landsat_Median_Composite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Python Modules

Import the Python modules needed for this notebook.

In [1]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

## Install pb_gee_tools

The pb_gee_tools module (https://github.com/remotesensinginfo/pb_gee_tools) is a collection of tools used to make it easier to access datasets on Google Earth Engine.

The code below tests whether the module is available to import and if not will install the module from code repository.

In [2]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets

Cloning into 'pb_gee_tools'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 149 (delta 83), reused 113 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (149/149), 55.02 KiB | 9.17 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Processing ./pb_gee_tools
  Preparing metadata (setup.py) ... done
  Created wheel for pb_gee_tools: filename=pb_gee_tools-0.1.0-py3-none-any.whl size=16974 sha256=81e9eada1631ad67addf52f29fd84ae66b48ed97bdf170809dc24e87db560a91
  Stored in directory: /tmp/pip-ephem-wheel-cache-54_9qap7/wheels/e8/da/17/69dc01c6cbd07adb00923d8c1dd9a22fb31e96a17b74c93812
Successfully built pb_gee_tools


# Autheticate with Google Earth Engine

The next stage is to specify the project being used for your analysis and to autheticate with Google Earth Engine.

The first time you run the following cell it will come up with a prompt asking your permission to access your GEE account and project.

In [3]:
ee.Authenticate()
ee.Initialize(project="ee-pb-dev")  # <==== Replace this with your own EE project string

# Mount Google Drive

To access files within your Google Drive we need to 'mount it' to this colab environment.

Again, it will come up with a prompt asking your permission.

In [4]:
drive.mount("/content/drive")

Mounted at /content/drive


# Customise Inputs

This next section is where you need to specify input variables to control the Landsat image composite which is created.

You need to specify:

*   The path (on your google drive) to the vector layer specifying your region of interest. The vector layer can be any format that Geopandas is able to read (e.g., Shapefile, GeoJSON, GPKG).
*   The start date (i.e., earlier date) for the image search
*   The end date (i.e., later date) for the image search
*   The output no date value used - should always be 0
*   The output image file name (Note, do not specify the file extention - it will have .tif added when created)
*   The output directory on your google drive where the output image will be saved.



In [5]:
# The file path on google drive for ROI file.
vec_file = "/content/drive/MyDrive/test_shp/test_roi_poly.shp"

# Start and End date
start_date = datetime.datetime(year=2010, month=1, day=1)
end_date = datetime.datetime(year=2010, month=12, day=31)

# Output no data value
out_no_data_val = 0.0

# Output Landsat composite file name
out_file_name = "landsat_2010_med_comp"

# Output Directory on Google Earth Engine
out_gdrive_dir = "test_gee_outs"

# Processing Stages

In [6]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [7]:
# Get the Landsat image collection
ls_img_col = pb_gee_tools.datasets.get_sr_landsat_collection(
    aoi=tile_aoi,
    start_date=start_date,
    end_date=end_date,
    cloud_thres=70,
    ignore_ls7=False,
    out_lstm_bands=True,
)

# Create a median composite
ls_median = (
    ls_img_col.median()
    .unmask(out_no_data_val)
    .clip(tile_aoi)
    .select(["Blue", "Green", "Red", "NIR", "SWIR1", "SWIR2"])
    .toUint16()
)

In [8]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Add ESRI Satellite Imagery Basemap
esri_img_url = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
map_obj.add_tile_layer(esri_img_url, name="ESRI Imagery", attribution="ESRI")

# Specify the visualisation options.
vis_params = {"min": 0, "max": 3000, "bands": ["NIR", "SWIR1", "Red"]}

# Add the Landsat Composite
map_obj.addLayer(ls_median, vis_params, "Landsat-8")

map_obj

Map(center=[-25.673248008010564, 147.43850893743968], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
# Export the median image to your Google Drive
task = ee.batch.Export.image.toDrive(
    image=ls_median,
    description=out_file_name,
    folder=out_gdrive_dir,
    scale=30, # export 30 m resolution image
    fileFormat="GeoTIFF",
    formatOptions={"cloudOptimized": True, "noData": out_no_data_val},
)
task.start()